In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

print("Python version: ", sys.version)
print("Numpy version: ", np.__version__)
print("Tensorflow version: ", tf.__version__)

Python version:  3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:25:59) 
[GCC 10.3.0]
Numpy version:  1.22.3
Tensorflow version:  2.7.0


In [2]:
# Loading data
def load_asl(verbose=False):
    # Data directories
    train_dir = 'processed_data/train'
    val_dir = 'processed_data/val'
    test_dir = 'processed_data/test'

    img_height = 200
    img_width = 200
    batch_size = 64

    train_datagenerator = ImageDataGenerator(rescale = 1./255)
    val_datagenerator = ImageDataGenerator(rescale = 1./255)
    test_datagenerator = ImageDataGenerator(rescale = 1./255)

    train_generator = train_datagenerator.flow_from_directory(
        train_dir,
        target_size = (img_height, img_width),
        batch_size = batch_size,
        class_mode = 'categorical',
    )

    val_generator = val_datagenerator.flow_from_directory(
        val_dir,
        target_size = (img_height, img_width),
        class_mode = 'categorical',
        batch_size = batch_size,
    )

    test_generator = test_datagenerator.flow_from_directory(
        test_dir,
        target_size = (img_height, img_width),
        class_mode = 'categorical',
        shuffle = False
    )

    return train_generator, val_generator, test_generator

In [3]:
train_generator, val_generator, test_generator = load_asl()

Found 52200 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.
Found 17428 images belonging to 29 classes.


In [4]:
# Import and freeze VGG model 
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='max')
base_model.trainable = False

# Add custom top layers
inputs = Input(shape=(224,224,3))
feature_maps = base_model(inputs, training=False)

dense_layer_1 = Dense(512, activation='leaky_relu', kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(feature_maps)
dropout_1 = Dropout(0.2)(dense_layer_1)
dense_layer_2 = Dense(256, activation='leaky_relu', kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(dropout_1)
dropout_2 = Dropout(0.2)(dense_layer_2)
dense_layer_3 = Dense(128, activation='leaky_relu', kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(dropout_2)
dropout_3 = Dropout(0.2)(dense_layer_3)
predictions = Dense(29, activation='softmax',kernel_regularizer=l2(0.0001))(dropout_3)

model = Model(inputs=inputs, outputs=predictions)

2024-04-23 15:28:44.767631: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 15:28:45.607864: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78911 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:0f:00.0, compute capability: 8.0


In [5]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.compile(optimizer=Nadam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    train_generator,
    batch_size = 64,
    epochs= 50,
    validation_data=(val_generator),
    shuffle=True,
    callbacks=[early_stopping]
)

Epoch 1/50


2024-04-23 15:28:48.387629: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201


  3/816 [..............................] - ETA: 49s - loss: 4.2297 - accuracy: 0.0469  

2024-04-23 15:28:51.280404: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


816/816 [==============================] - 121s 143ms/step - loss: 1.0806 - accuracy: 0.7306 - val_loss: 0.4245 - val_accuracy: 0.9318
Epoch 2/50
816/816 [==============================] - 111s 137ms/step - loss: 0.4844 - accuracy: 0.9061 - val_loss: 0.3801 - val_accuracy: 0.9404
Epoch 3/50
816/816 [==============================] - 116s 142ms/step - loss: 0.4264 - accuracy: 0.9300 - val_loss: 0.2893 - val_accuracy: 0.9729
Epoch 4/50
816/816 [==============================] - 112s 138ms/step - loss: 0.3594 - accuracy: 0.9462 - val_loss: 0.3123 - val_accuracy: 0.9621
Epoch 5/50
816/816 [==============================] - 109s 133ms/step - loss: 0.3399 - accuracy: 0.9520 - val_loss: 0.2859 - val_accuracy: 0.9689
Epoch 6/50
816/816 [==============================] - 113s 139ms/step - loss: 0.3260 - accuracy: 0.9552 - val_loss: 0.2859 - val_accuracy: 0.9663
Epoch 7/50
816/816 [==============================] - 109s 134ms/step - loss: 0.3049 - accuracy: 0.9609 - val_loss: 0.2494 - val_accura

In [6]:
model.save('vgg_model.keras')

In [9]:
print('Test Accuracy: ', model.evaluate(test_generator)[1])

545/545 [==============================] - 29s 54ms/step - loss: 0.1900 - accuracy: 0.9915
Test Accuracy:  0.9915079474449158
